In [271]:
import pandas as pd
def prosFomularioAdicional(filepath):

    xls = pd.ExcelFile(filepath)
    dfs = {}
    for sheet_name in xls.sheet_names:
        dfs[sheet_name] = pd.read_excel(xls, sheet_name)
    return dfs

def prosConsolidado(filepath):
    xls = pd.ExcelFile(filepath)
    sheet_names = xls.sheet_names
    result_dfs = {}

    for sht in sheet_names:
        df = pd.read_excel(filepath, sheet_name=sht)

        # Evaluar si la hoja es la mensual o es de especialidad, 
        #** Agregar validaciones adicionales para asegurar que solo se procese el file correcto
        if sht == "Mensual":   
            tipo_hoja = "Mensual"
        else:
            tipo_hoja = "Especialidad"

        # Eliminar filas y columnas vacías

        df.dropna(axis = 0,how = "all",inplace = True)  
        df.dropna(axis = 1,how = "all",inplace = True)  
        df = df.reset_index(drop=True)

        # Eliminar fila extra de la hoja "Mensual"

        if tipo_hoja == "Mensual":
            df.drop(2, inplace = True)
            df = df.reset_index(drop=True)

        # La hoja "Especialidad" tiene una columna adicional que permite identificar la tabla donde se encuentra el agregado de datos 
        # Se eliminan las otras filas y se actualiza el df al tamaño correcto para procesarlo

        elif tipo_hoja == "Especialidad":
            indexnum = int(df.loc[df.iloc[:,0] == "TOTAL"].index[0])

            df = df.iloc[indexnum:]

            nombre_esp = df.iloc[0,1]
            codigo_esp = sht  #depende de cada hoja
            df = df.iloc[:,1:] 

        # Generar una tabla(Posible serie), de las horas contratadas, programadas, etc.
        # Se hace un slice de las primeras dos filas, y se eliminan vacios 

        df_jornada = df.iloc[1:3,1:8].copy() 
        df_jornada.dropna(axis = 1,how = "all",inplace = True)
        df_jornada.columns = df_jornada.iloc[0]
        df_jornada = df_jornada[1:]
        df_jornada = df_jornada.reset_index(drop=True)
        df_jornada.columns.rename("",inplace=True)
        df_jornada = df_jornada.transpose()
        df_jornada.rename(columns={0: 'Horas'},inplace= True)

        # Replicar la tabla de excel con las horas utilizadas por actividad
        # Se hace un slice de las primeras filas 3 a 17  , se eliminan vacios y se nombran columnas de acuerdo al excel

        df_tiempo = df.iloc[3:18].copy()
        df_tiempo.dropna(axis = 1,how = "all",inplace = True)
        df_tiempo.dropna(axis = 0,how = "all",inplace = True)
        df_tiempo.columns = ['Actividad', 'Programado', 'Utilizado', 'N° Actividades']
        df_tiempo = df_tiempo.reset_index(drop=True)

        # Replicar la tabla de excel del resumen del mes con los totales de consultas por cada categoria

        # Se hace un slice de las primeras filas 18 en adelante, se eliminan vacios full y se auto completan algunas categorias faltantes
        df_total = df.iloc[18:].copy()
        df_total.dropna(axis = 1,how = "all",inplace = True)
        df_total[:3].fillna(method="ffill", axis=1,inplace = True)
        df_total[:3].fillna(method="ffill", axis=0,inplace = True)

        # Se transpone la tabla para concatenar las categorias y unificarlas en una sola columna(celdas combinadas de excel) 
        df_total = df_total.transpose()
        df_total.iloc[1, 1] = df_total.iloc[1, 1] +" - "+ df_total.iloc[1, 2]
        df_total.iloc[2:6, 1] = df_total.iloc[2:6, 1].astype(str) +" - "+ df_total.iloc[2:6, 2].astype(str) +" - "+ df_total.iloc[2:6, 3].astype(str)
        df_total.iloc[1:-3, 0] = df_total.iloc[1:-3, 0].astype(str) +" - "+ df_total.iloc[1:-3, 1].astype(str)

        #Se eliminan las columnas que tenian las subcategorias, y se reindexa
        df_total.drop(df_total.columns[[1, 2, 3]],axis = 1, inplace = True)
        df_total.columns = df_total.iloc[0]
        df_total = df_total[1:]
        df_total = df_total.reset_index(drop=True)
        df_total.columns.rename("",inplace=True)
        df_total.iloc[:, 0] = df_total.iloc[:, 0].str.replace('Frecuencia (marcar solo una)', 'Frecuencia', regex=False)

        # Se transpone la tabla para volver al formato original (validar si vale la pena)
        df_total = df_total.transpose()
        df_total.columns = df_total.iloc[0]
        df_total = df_total[1:]

        result_dfs[sht] = {
            'df_jornada': df_jornada,
            'df_tiempo': df_tiempo,
            'df_total': df_total
        }
    return result_dfs


In [272]:
formAdicional = 'Adicionales_con datos.xlsx'
hojasFormulario = prosFomularioAdicional(formAdicional)

c:\Users\anjos\Anaconda3\envs\Curso_Udemy\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
c:\Users\anjos\Anaconda3\envs\Curso_Udemy\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
c:\Users\anjos\Anaconda3\envs\Curso_Udemy\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [273]:
consolidado = "Muestra consolidado.xlsx"
hojasConsolidado = prosConsolidado(consolidado)

In [278]:
periodo = "Febrero 2023"

In [277]:
#Calculo oficial de indicadores servicio
indServicio = pd.DataFrame(columns=['PERIODO', 'INDICADOR', 'VALOR'])

nuevoIndServ=[]

nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'ESTANDAR PACIENTE POR HORA DE LA UNIDAD',
    'VALOR': (hojasFormulario['Consultas Externas']['Consultas programadas en consulta externa'].sum()/hojasFormulario['Consultas Externas']['Horas programadas para consulta externa'].sum())
             if hojasFormulario['Consultas Externas']['Horas programadas para consulta externa'].sum() > 0 else None 
})

nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'CANTIDAD DE PRÓTESIS REALIZADAS TOTAL SUPERIOR',
    'VALOR': hojasFormulario['Otros Datos']['Resultado'][0]
})

nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'CANTIDAD DE PRÓTESIS REALIZADAS TOTAL INFERIOR',
    'VALOR': hojasFormulario['Otros Datos']['Resultado'][1]
})

nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'CANTIDAD DE PRÓTESIS REALIZADAS PARCIAL SUPERIOR',
    'VALOR': hojasFormulario['Otros Datos']['Resultado'][2]
})

nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'CANTIDAD DE PRÓTESIS REALIZADAS PARCIAL INFERIOR',
    'VALOR': hojasFormulario['Otros Datos']['Resultado'][3]
})

nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'CANTIDAD DE PRÓTESIS REALIZADAS OBTURADORES',
    'VALOR': hojasFormulario['Otros Datos']['Resultado'][4]
})

nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'CANTIDAD DE PRÓTESIS REALIZADAS REPARACIONES',
    'VALOR': hojasFormulario['Otros Datos']['Resultado'][5]
})

nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'TOTAL DE APARATOS ORTODONCIA',
    'VALOR': hojasFormulario['Otros Datos']['Resultado'][6]
})

nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'TOTAL DE APARATOS ODONTOPEDIATRIA',
    'VALOR': hojasFormulario['Otros Datos']['Resultado'][7]
})

nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'TOTAL DE PLANOS OCLUSALES',
    'VALOR': hojasFormulario['Otros Datos']['Resultado'][8]
})

nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'CANTIDAD DE REFERENCIAS ACEPTADAS',
    'VALOR': hojasFormulario['Referencias']['Aceptado'].sum()
})

nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'CANTIDAD DE REFERENCIAS RECHAZADAS',
    'VALOR': hojasFormulario['Referencias']['Rechazado'].sum()
})

nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'CONSULTAS ODONTOLÓGICAS PRIMERA VEZ',
    'VALOR': hojasConsolidado["Mensual"]["df_total"].iloc[0,:5].sum()
})

nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'CONSULTAS ODONTOLÓGICAS SUBSECUENTES',
    'VALOR': hojasConsolidado["Mensual"]["df_total"].iloc[0,5]
})

nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'TELECONSULTAS',
    'VALOR': hojasFormulario['Otros Datos']['Resultado'][9]
})

nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'HORAS EN TELECONSULTAS',
    'VALOR': hojasFormulario['Otros Datos']['Resultado'][10]
})

nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'TELEORIENTACIONES',
    'VALOR': hojasFormulario['Otros Datos']['Resultado'][11]
})

nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'HORAS EN TELEORIENTACIONES',
    'VALOR': hojasFormulario['Otros Datos']['Resultado'][12]
})

nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'APARATOLOGÍA',
    'VALOR': hojasFormulario['Otros Datos']['Resultado'][13]
})

nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'HORAS DE HOSPITALIZACIÓN',
    'VALOR': hojasFormulario['Otros Datos']['Resultado'][14]
})

nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'NÚMERO DE NIÑOS (AS) DE 0 A MENOS DE 10 AÑOS CON ATENCIÓN ODONTOLÓGICA PREVENTIVA DE PRIMERA VEZ EN EL AÑO',
    'VALOR': hojasConsolidado["Mensual"]["df_total"].iloc[1,7]
})

nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'NÚMERO DE ADOLESCENTES DE 10 A MENOS DE 20 AÑOS CON ATENCIÓN ODONTOLÓGICA PREVENTIVA DE PRIMERA VEZ EN EL AÑO',
    'VALOR': hojasConsolidado["Mensual"]["df_total"].iloc[2,7]
})

nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'PACIENTES EMBARAZADAS CON ATENCIÓN ODONTOLÓGICA PREVENTIVA DE PRIMERA VEZ EN EL AÑO',
    'VALOR': hojasConsolidado["Mensual"]["df_total"].iloc[0,6]
})

nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'COBERTURA ODONTOLÓGICA EN NIÑOS (AS) DE 0 A MENOS DE 10 AÑOS, EN EL TERCER NIVEL DE ATENCIÓN',
    'VALOR': (hojasConsolidado["Mensual"]["df_total"].iloc[1,[0,1,3]].sum()/hojasFormulario['Otros Datos']['Resultado'][17])
             if hojasFormulario['Otros Datos']['Resultado'][17] > 0 else None
})

nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'COBERTURA ODONTOLÓGICA EN ADOLESCENTES DE 10 A MENOS DE 20 AÑOS, EN EL TERCER NIVEL DE ATENCIÓN',
    'VALOR': (hojasConsolidado["Mensual"]["df_total"].iloc[2,[0,1,3]].sum()/hojasFormulario['Otros Datos']['Resultado'][18])
             if hojasFormulario['Otros Datos']['Resultado'][18] > 0 else None
})

nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'COBERTURA ODONTOLÓGICA EN HOMBRES DE 20 AÑOS A 64 AÑOS, EN EL TERCER NIVEL DE ATENCIÓN',
    'VALOR': (hojasConsolidado["Mensual"]["df_total"].iloc[3,[0,1,3]].sum()/hojasFormulario['Otros Datos']['Resultado'][19])
             if hojasFormulario['Otros Datos']['Resultado'][19] > 0 else None
})


nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'COBERTURA ODONTOLÓGICA EN MUJERES DE 20 AÑOS A 64 AÑOS, EN TERCER NIVEL DE ATENCIÓN',
    'VALOR': (hojasConsolidado["Mensual"]["df_total"].iloc[4,[0,1,3]].sum()/hojasFormulario['Otros Datos']['Resultado'][20])
             if hojasFormulario['Otros Datos']['Resultado'][20] > 0 else None
})


nuevoIndServ.append({
    'PERIODO': periodo,
    'INDICADOR': 'COBERTURA ODONTOLÓGICA EN PERSONAS DE MÁS DE 65 AÑOS, EN EL TERCER NIVEL DE ATENCIÓN',
    'VALOR': (hojasConsolidado["Mensual"]["df_total"].iloc[5,[0,1,3]].sum()/hojasFormulario['Otros Datos']['Resultado'][21])
             if hojasFormulario['Otros Datos']['Resultado'][21] > 0 else None
})
indServicio = pd.concat([indServicio, pd.DataFrame(nuevoIndServ)], ignore_index=True)
indServicio

,PERIODO,INDICADOR,VALOR
0,Febero 2023,ESTANDAR PACIENTE POR HORA DE LA UNIDAD,1.263158
1,Febero 2023,CANTIDAD DE PRÓTESIS REALIZADAS TOTAL SUPERIOR,3.000000
2,Febero 2023,CANTIDAD DE PRÓTESIS REALIZADAS TOTAL INFERIOR,4.000000
3,Febero 2023,CANTIDAD DE PRÓTESIS REALIZADAS PARCIAL SUPERIOR,5.000000
4,Febero 2023,CANTIDAD DE PRÓTESIS REALIZADAS PARCIAL INFERIOR,2.000000
5,Febero 2023,CANTIDAD DE PRÓTESIS REALIZADAS OBTURADORES,1.000000
6,Febero 2023,CANTIDAD DE PRÓTESIS REALIZADAS REPARACIONES,6.000000
7,Febero 2023,TOTAL DE APARATOS ORTODONCIA,20.000000
8,Febero 2023,TOTAL DE APARATOS ODONTOPEDIATRIA,15.000000
9,Febero 2023,TOTAL DE PLANOS OCLUSALES,10.000000


In [279]:
# Calculo oficial de indicadores especialidad
esps = hojasFormulario['Consultas Externas']['Especialidad'].unique()

indEspecialidad = pd.DataFrame(columns=['PERIODO', 'ESPECIALIDAD', 'INDICADOR', 'VALOR'])

for esp in esps:
    nuevoIndEsp = []

    nuevoIndEsp.append({
        'PERIODO': periodo,
        'ESPECIALIDAD': esp,
        'INDICADOR': 'HORAS PROGRAMADAS PARA LA ATENCION DE PACIENTES',
        'VALOR': hojasFormulario['Consultas Externas'][hojasFormulario['Consultas Externas']['Especialidad'] == esp]['Horas programadas para consulta externa'].sum()
    })

    nuevoIndEsp.append({
        'PERIODO': periodo,
        'ESPECIALIDAD': esp,
        'INDICADOR': 'PRODUCCIÓN REAL',
        'VALOR': hojasFormulario['Consultas Externas'][hojasFormulario['Consultas Externas']['Especialidad'] == esp]['Consultas realizadas en consulta externa'].sum()
    })

    nuevoIndEsp.append({
        'PERIODO': periodo,
        'ESPECIALIDAD': esp,
        'INDICADOR': 'HORAS EJECUTADAS EN LA ATENCION DE PACIENTES',
        'VALOR': hojasFormulario['Consultas Externas'][hojasFormulario['Consultas Externas']['Especialidad'] == esp]['Horas utilizadas para consulta externa'].sum()
    })

    nuevoIndEsp.append({
        'PERIODO': periodo,
        'ESPECIALIDAD': esp,
        'INDICADOR': 'USUARIOS ATENDIDOS POR HORA PROGRAMADO PARA LA ATENCION DE PACIENTES',
        'VALOR': (hojasFormulario['Consultas Externas'][hojasFormulario['Consultas Externas']['Especialidad'] == esp]['Consultas realizadas en consulta externa'].sum() /
                  hojasFormulario['Consultas Externas'][hojasFormulario['Consultas Externas']['Especialidad'] == esp]['Horas programadas para consulta externa'].sum())
                 if hojasFormulario['Consultas Externas'][hojasFormulario['Consultas Externas']['Especialidad'] == esp]['Horas programadas para consulta externa'].sum() > 0 else None
    })

    indEspecialidad = pd.concat([indEspecialidad, pd.DataFrame(nuevoIndEsp)], ignore_index=True)

indEspecialidad

,PERIODO,ESPECIALIDAD,INDICADOR,VALOR
0,Febrero 2023,O.G.A,HORAS PROGRAMADAS PARA LA ATENCION DE PACIENTES,30.000000
1,Febrero 2023,O.G.A,PRODUCCIÓN REAL,60.000000
2,Febrero 2023,O.G.A,HORAS EJECUTADAS EN LA ATENCION DE PACIENTES,30.000000
3,Febrero 2023,O.G.A,USUARIOS ATENDIDOS POR HORA PROGRAMADO PARA LA...,2.000000
4,Febrero 2023,O.G.,HORAS PROGRAMADAS PARA LA ATENCION DE PACIENTES,65.000000
5,Febrero 2023,O.G.,PRODUCCIÓN REAL,48.000000
6,Febrero 2023,O.G.,HORAS EJECUTADAS EN LA ATENCION DE PACIENTES,65.000000
7,Febrero 2023,O.G.,USUARIOS ATENDIDOS POR HORA PROGRAMADO PARA LA...,0.738462


In [288]:
#Calculo oficial de indicadores profesional
indDoctor = pd.DataFrame(columns=['PERIODO','PROFESIONAL', 'INDICADOR', 'VALOR'])

for index, row in hojasFormulario['Consultas Externas'].iterrows():
    nuevoIndDoc = []
	
    nuevoIndDoc.append({
        'PERIODO': periodo,
        'PROFESIONAL' : row['Profesional'],
        'INDICADOR': 'CAPACIDAD MAXIMA PRÁCTICA',
        'VALOR': ( 2 * row['Horas programadas para consulta externa'])
    })

    nuevoIndDoc.append({
        'PERIODO': periodo,
        'PROFESIONAL' : row['Profesional'],
        'INDICADOR': 'CAPACIDAD DE PRODUCCIÓN PREVISTA',
        'VALOR': ( row['Horas programadas para consulta externa'] * hojasFormulario['Metas']['Meta'][2])
    })

    nuevoIndDoc.append({
        'PERIODO': periodo,
        'PROFESIONAL' : row['Profesional'],
        'INDICADOR': 'PRODUCCIÓN REAL',
        'VALOR': row['Consultas realizadas en consulta externa']
    })

    nuevoIndDoc.append({
        'PERIODO': periodo,
        'PROFESIONAL' : row['Profesional'],
        'INDICADOR': 'USUARIOS ATENDIDOS POR HORA PROGRAMADO PARA LA ATENCION DE PACIENTES',
        'VALOR': (row['Consultas realizadas en consulta externa']/row['Horas programadas para consulta externa'])
                 if row['Horas programadas para consulta externa'] > 0 else None
    })

    nuevoIndDoc.append({
        'PERIODO': periodo,
        'PROFESIONAL' : row['Profesional'],
        'INDICADOR': 'USUARIOS POR HORA UTILIZADA EN ATENCION DE PACIENTES',
        'VALOR': (row['Consultas realizadas en consulta externa']/row['Horas utilizadas para consulta externa'])
                 if row['Horas utilizadas para consulta externa'] > 0 else None
    })

    nuevoIndDoc.append({
        'PERIODO': periodo,
        'PROFESIONAL' : row['Profesional'],
        'INDICADOR': 'AUSENTISMO CONSULTA EXTERNA',
        'VALOR': (row['Citas perdidas en consulta externa']/(row['Consultas realizadas en consulta externa']+row['Citas perdidas en consulta externa']+row['Cupos no utilizados en consulta externa']))
                 if (row['Consultas realizadas en consulta externa']+row['Citas perdidas en consulta externa']+row['Cupos no utilizados en consulta externa']) > 0 else None
    })

    nuevoIndDoc.append({
        'PERIODO': periodo,
        'PROFESIONAL' : row['Profesional'],
        'INDICADOR': 'SUSTITUCIÓN DE PACIENTES CONSULTA EXTERNA',
        'VALOR': (row['Citas sustituidas en consulta externa']/row['Citas perdidas en consulta externa'])
                 if row['Citas perdidas en consulta externa'] > 0 else None
    })

    balPerExt = ((row['Citas perdidas en consulta externa']+row['Cupos no utilizados en consulta externa']) - (row['Citas sustituidas en consulta externa']+row['Recargos en consulta externa']))
   
    nuevoIndDoc.append({
        'PERIODO': periodo,
        'PROFESIONAL' : row['Profesional'],
        'INDICADOR': 'BALANCE DE CUPOS PERDIDOS EN CONSULTA EXTERNA',
        'VALOR': balPerExt 
    })

    nuevoIndDoc.append({
        'PERIODO': periodo,
        'PROFESIONAL' : row['Profesional'],
        'INDICADOR': 'COSTO POR CUPOS PERDIDOS EN CONSULTA EXTERNA',
        'VALOR': (balPerExt * hojasFormulario['Otros Datos']['Resultado'][15]) 
                
    })
    indDoctor = pd.concat([indDoctor, pd.DataFrame(nuevoIndDoc)], ignore_index=True)
	
#Consulta Procedimientos
for index, row in hojasFormulario['Consultas Procedimientos'].iterrows():
	nuevoIndDoc = []
	nuevoIndDoc.append({
        'PERIODO': periodo,
        'PROFESIONAL' : row['Profesional'],
        'INDICADOR': 'AUSENTISMO CONSULTA PROCEDIMIENTOS',
        'VALOR': (row['Citas perdidas en consulta procedimiento']/(row['Consultas realizadas en consulta procedimiento']+row['Citas perdidas en consulta procedimiento']+row['Cupos no utilizados en consulta procedimiento']))
                 if (row['Consultas realizadas en consulta procedimiento']+row['Citas perdidas en consulta procedimiento']+row['Cupos no utilizados en consulta procedimiento']) > 0 else None
    })
	nuevoIndDoc.append({
        'PERIODO': periodo,
        'PROFESIONAL' : row['Profesional'],
        'INDICADOR': 'SUSTITUCIÓN DE PACIENTES CONSULTA PROCEDIMIENTOS',
        'VALOR': (row['Citas sustituidas en consulta procedimiento']/row['Citas perdidas en consulta procedimiento'])
                 if row['Citas perdidas en consulta procedimiento'] > 0 else None
    })
	balPerProc = ((row['Citas perdidas en consulta procedimiento']+row['Cupos no utilizados en consulta procedimiento'])-(row['Citas sustituidas en consulta procedimiento']+row['Recargos en consulta procedimiento']))
	nuevoIndDoc.append({
        'PERIODO': periodo,
        'PROFESIONAL' : row['Profesional'],
        'INDICADOR': 'BALANCE DE CUPOS PERDIDOS EN CONSULTA PROCEDIMIENTOS',
        'VALOR': balPerProc
    })
	nuevoIndDoc.append({
        'PERIODO': periodo,
        'PROFESIONAL' : row['Profesional'],
        'INDICADOR': 'COSTO POR CUPOS PERDIDOS EN CONSULTA PROCEDIMIENTOS',
        'VALOR': (balPerProc * hojasFormulario['Otros Datos']['Resultado'][16])
    })
	indDoctor = pd.concat([indDoctor, pd.DataFrame(nuevoIndDoc)], ignore_index=True)

#Consulta Procedimientos
for index, row in hojasFormulario['Ortodoncia-Ortopedia'].iterrows():
	nuevoIndDoc = []
	nuevoIndDoc.append({
        'PERIODO': periodo,
        'PROFESIONAL' : row['Profesional'],
        'INDICADOR': 'PORCENTAJE DE PACIENTES EN ORTODONCIA',
        'VALOR': row['Porcentaje de pacientes en ortodoncia']
    })
	nuevoIndDoc.append({
        'PERIODO': periodo,
        'PROFESIONAL' : row['Profesional'],
        'INDICADOR': 'PORCENTAJE DE PACIENTES EN ORTOPEDIA',
        'VALOR': row['Porcentaje de pacientes en ortopedia']
    })
	indDoctor = pd.concat([indDoctor, pd.DataFrame(nuevoIndDoc)], ignore_index=True)
    

indDoctor['original_index'] = indDoctor.index
indDoctor = indDoctor.sort_values(by=['PROFESIONAL', 'original_index'])
indDoctor.drop('original_index', axis=1, inplace=True)
indDoctor

,PERIODO,PROFESIONAL,INDICADOR,VALOR
0,Febrero 2023,Andres,CAPACIDAD MAXIMA PRÁCTICA,40.000000
1,Febrero 2023,Andres,CAPACIDAD DE PRODUCCIÓN PREVISTA,1200.000000
2,Febrero 2023,Andres,PRODUCCIÓN REAL,40.000000
3,Febrero 2023,Andres,USUARIOS ATENDIDOS POR HORA PROGRAMADO PARA LA...,2.000000
4,Febrero 2023,Andres,USUARIOS POR HORA UTILIZADA EN ATENCION DE PAC...,2.222222
...,...,...,...,...
54,Febrero 2023,Yos,SUSTITUCIÓN DE PACIENTES CONSULTA PROCEDIMIENTOS,1.666667
55,Febrero 2023,Yos,BALANCE DE CUPOS PERDIDOS EN CONSULTA PROCEDIM...,-4.000000
56,Febrero 2023,Yos,COSTO POR CUPOS PERDIDOS EN CONSULTA PROCEDIMI...,-157.880000
69,Febrero 2023,Yos,PORCENTAJE DE PACIENTES EN ORTODONCIA,0.655660


In [267]:
#Referencias por area de salud
indReferencias = hojasFormulario['Referencias'].iloc[:,1:].groupby('Área de Salud', as_index = False).sum()
indReferencias.insert(0, 'PERIODO', periodo)
indReferencias

,PERIODO,Área de Salud,O.G.,O.G.A,ORTOD.,ENDOD.,PERIOD.,PROSTOD.,TTM D.O.,PROT.MAXILOF.,ODONTOPED.,ODONTOGER.,CIR.MAXILOF.,Rechazado,Aceptado
0,Febero 2023,Abangares,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
1,Febero 2023,Acosta,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,Febero 2023,Aguas Zarcas,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,Febero 2023,Alajuela Central,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,4.0
4,Febero 2023,Alajuela Norte,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Febero 2023,Turrialba-Jiménez,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
60,Febero 2023,Upala,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
61,Febero 2023,Valle La Estrella,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0
62,Febero 2023,Valverde Vega,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


In [268]:
#Listas de espera

indListas = hojasFormulario['Listas de espera'].copy()
indListas.insert(0, 'PERIODO', periodo)
indListas

,PERIODO,Especialidad,Factor crítico,Fecha próxima cita
0,Febero 2023,O.G.,Proc 1,2023-12-12
1,Febero 2023,O.G.A,Proc 2,2023-12-17
2,Febero 2023,ORTOD.,Proc 3,2023-12-17
3,Febero 2023,ENDOD.,Proc 4,2023-12-12
4,Febero 2023,PERIOD.,Proc 5,2023-12-17
5,Febero 2023,PROSTOD.,Proc 6,2023-12-17
6,Febero 2023,TTM D.O.,Proc 7,2024-01-07
7,Febero 2023,PROT.MAXILOF.,Proc 8,2023-12-12
8,Febero 2023,ODONTOPED.,Proc 9,2023-12-17
9,Febero 2023,ODONTOGER.,Proc 10,2023-12-17


In [269]:
#Listas de espera
listMetas = hojasFormulario['Metas'].copy()
listMetas.insert(0, 'PERIODO', periodo)
listMetas


,PERIODO,Indicador,Meta,Rango,Tipo de meta
0,Febero 2023,CAPACIDAD MAXIMA PRÁCTICA,NaN,100.0,Logro
1,Febero 2023,HORAS PROGRAMADAS PARA LA ATENCION DE PACIENTES,50.0,200.0,Reducción
2,Febero 2023,ESTANDAR PACIENTE POR HORA DE LA UNIDAD,60.0,NaN,Logro
3,Febero 2023,CAPACIDAD DE PRODUCCIÓN PREVISTA,NaN,NaN,Reducción
4,Febero 2023,PRODUCCIÓN REAL,NaN,NaN,NaN
5,Febero 2023,HORAS UTILIZADAS EN LA ATENCION DE PACIENTES,NaN,NaN,NaN
6,Febero 2023,USUARIOS ATENTIDOS POR HORA PROGRAMADA PARA LA...,NaN,NaN,NaN
7,Febero 2023,USUARIOS ATENDIDOS POR HORA UTILIZADA EN ATENC...,NaN,NaN,NaN
8,Febero 2023,AUSENTISMO EN CONSULTA EXTERNA,NaN,NaN,NaN
9,Febero 2023,AUSENTISMO EN CONSULTA PROCEDIMIENTOS,NaN,NaN,NaN


In [270]:
#Cargar tablas a BD
import sqlite3

sqlconn = sqlite3.connect('PruebasTFG.db')
cursor = sqlconn.cursor()

indServicio.to_sql('IndicadoresServicio', sqlconn, if_exists='append', index=False)
indEspecialidad.to_sql('IndicadoresEspecialidad', sqlconn, if_exists='append', index=False)
indDoctor.to_sql('IndicadoresDoctor', sqlconn, if_exists='append', index=False)
indReferencias.to_sql('IndicadoresReferencias', sqlconn, if_exists='append', index=False)
indListas.to_sql('IndicadoresListas', sqlconn, if_exists='append', index=False)
listMetas.to_sql('IndicadoresMetas', sqlconn, if_exists='append', index=False)

cursor.close()
sqlconn.close()


In [261]:
#Crear tablas, solo se hace una vez
import sqlite3

sqlconn = sqlite3.connect('PruebasTFG.db')
cursor = sqlconn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS IndicadoresServicio (
        ID INTEGER PRIMARY KEY AUTOINCREMENT,
        PERIODO TEXT,
        INDICADOR TEXT,
        VALOR REAL
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS IndicadoresEspecialidad (
        ID INTEGER PRIMARY KEY AUTOINCREMENT,
        PERIODO TEXT,
        ESPECIALIDAD TEXT,
        INDICADOR TEXT,
        VALOR REAL
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS IndicadoresDoctor (
        ID INTEGER PRIMARY KEY AUTOINCREMENT,
        PERIODO TEXT,
        PROFESIONAL TEXT,
        INDICADOR TEXT,
        VALOR REAL
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS IndicadoresReferencias (
        PERIODO	TEXT,
        "Área de Salud" TEXT,
        "O.G." REAL,
        "O.G.A" REAL,
        "ORTOD." REAL,
        "ENDOD." REAL,
        "PERIOD." REAL,
        "PROSTOD." REAL,
        "TTM D.O." REAL,
        "PROT.MAXILOF." REAL,
        "ODONTOPED." REAL,
        "ODONTOGER." REAL,
        "CIR.MAXILOF." REAL,
        Rechazado REAL,
        Aceptado REAL
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS IndicadoresListas (
        PERIODO	TEXT,
        Especialidad TEXT,
        "Factor crítico"	TEXT,
	    "Fecha próxima cita"	DATE
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS IndicadoresMetas (
        PERIODO TEXT,
        Indicador TEXT,
        Meta REAL,
        Rango REAL,
        "Tipo de meta" TEXT
    )
''')

cursor.close()
sqlconn.close()

In [253]:
#Drop tablas
import sqlite3

# Connect to your SQLite database
conn = sqlite3.connect('PruebasTFG.db')
cursor = conn.cursor()

# Retrieve a list of all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' and name not like ('%sqlite%');")
tables = cursor.fetchall()

# Iterate over the list of tables and drop each one
for table in tables:
    table_name = table[0]  # table name is in the first column
    cursor.execute(f"DROP TABLE IF EXISTS {table_name};")

# Commit changes and close the connection
conn.commit()
conn.close()